<a href="https://colab.research.google.com/github/Salva13s/HybridModel/blob/main/UnetAdapt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, density_map_dir, pseudo_map_dir=None, transform=None):
        self.image_dir = image_dir
        self.density_map_dir = density_map_dir
        self.pseudo_map_dir = pseudo_map_dir
        self.transform = transform

        self.image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
        self.density_map_files = sorted([f for f in os.listdir(density_map_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

        if pseudo_map_dir:
            self.pseudo_map_files = sorted([f for f in os.listdir(pseudo_map_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
        else:
            self.pseudo_map_files = None

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        density_map_path = os.path.join(self.density_map_dir, self.density_map_files[idx])

        image = Image.open(image_path).convert('RGB')
        density_map = Image.open(density_map_path).convert('L')

        if self.pseudo_map_dir:
            pseudo_map_path = os.path.join(self.pseudo_map_dir, self.pseudo_map_files[idx])
            pseudo_map = Image.open(pseudo_map_path).convert('L')
        else:
            pseudo_map = None

        if self.transform:
            image = self.transform(image)
            density_map = self.transform(density_map)
            if pseudo_map:
                pseudo_map = self.transform(pseudo_map)

        # Devolver también el nombre del archivo de imagen
        return image, density_map, pseudo_map, self.image_files[idx]

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
train_image_dir = '/content/drive/MyDrive/1 Tesis/Fish/DataDividido/train/reales_resized'
train_density_map_dir = '/content/drive/MyDrive/1 Tesis/Fish/DataDividido/train/generadas_resized'
train_pseudo_map_dir = '/content/drive/MyDrive/1 Tesis/Fish/DataDividido/train/PsMapas'

val_image_dir = '/content/drive/MyDrive/1 Tesis/Fish/DataDividido/val/reales_resized'
val_density_map_dir = '/content/drive/MyDrive/1 Tesis/Fish/DataDividido/val/generadas_resized'
val_pseudo_map_dir = '/content/drive/MyDrive/1 Tesis/Fish/DataDividido/val/PsMapas'

test_image_dir = '/content/drive/MyDrive/1 Tesis/Fish/DataDividido/test/reales'
test_density_map_dir = '/content/drive/MyDrive/1 Tesis/Fish/DataDividido/test/generadas_resized'
test_pseudo_map_dir = '/content/drive/MyDrive/1 Tesis/Fish/DataDividido/test/PsMapas'

train_dataset = CustomDataset(train_image_dir, train_density_map_dir, train_pseudo_map_dir, transform=transform)
val_dataset = CustomDataset(val_image_dir, val_density_map_dir, val_pseudo_map_dir, transform=transform)
test_dataset = CustomDataset(test_image_dir, test_density_map_dir, test_pseudo_map_dir, transform=transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=6, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=6, shuffle=False)

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()

        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            )

        def upconv_block(in_channels, out_channels):
            return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

        self.encoder1 = conv_block(in_channels, 64)
        self.encoder2 = conv_block(64, 128)
        self.encoder3 = conv_block(128, 256)
        self.encoder4 = conv_block(256, 512)
        self.encoder5 = conv_block(512, 1024)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.upconv4 = upconv_block(1024, 512)
        self.decoder4 = conv_block(1024, 512)
        self.upconv3 = upconv_block(512, 256)
        self.decoder3 = conv_block(512, 256)
        self.upconv2 = upconv_block(256, 128)
        self.decoder2 = conv_block(256, 128)
        self.upconv1 = upconv_block(128, 64)
        self.decoder1 = conv_block(128, 64)

        self.conv_final = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))
        enc4 = self.encoder4(self.pool(enc3))
        enc5 = self.encoder5(self.pool(enc4))

        dec4 = self.upconv4(enc5)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)

        return self.conv_final(dec1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_unet = UNet().to(device)

In [ ]:
class AdaptiveNet(nn.Module):
    def __init__(self, unet_output_channels=1):
        super(AdaptiveNet, self).__init__()

        self.conv1 = nn.Conv2d(unet_output_channels + 1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 1, kernel_size=1)
        self.relu = nn.ReLU()

    def forward(self, unet_output, pseudo_density_map):
        x = torch.cat((unet_output, pseudo_density_map), dim=1)
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return x

model_adaptive = AdaptiveNet().to(device)

In [ ]:
def train_jointly(unet_model, adaptive_model, train_loader, val_loader, num_epochs=50):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(list(unet_model.parameters()) + list(adaptive_model.parameters()), lr=1e-4)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

    unet_model.to(device)
    adaptive_model.to(device)

    train_loss_history = []
    val_loss_history = []

    for epoch in range(num_epochs):
        unet_model.train()
        adaptive_model.train()

        running_loss = 0.0

        # Entrenamiento
        for (images, density_maps, pseudo_maps, _) in train_loader:
            images, density_maps = images.to(device), density_maps.to(device)
            pseudo_maps = pseudo_maps.to(device)

            optimizer.zero_grad()

            unet_output = unet_model(images)
            adaptive_output = adaptive_model(unet_output, pseudo_maps)

            loss = criterion(adaptive_output, density_maps)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss_train = running_loss / len(train_loader.dataset)
        train_loss_history.append(epoch_loss_train)

        # Pérdida de validación
        unet_model.eval()
        adaptive_model.eval()
        running_loss_val = 0.0

        with torch.no_grad():
            for (images, density_maps, pseudo_maps, _) in val_loader:
                images, density_maps = images.to(device), density_maps.to(device)
                pseudo_maps = pseudo_maps.to(device)

                unet_output = unet_model(images)
                adaptive_output = adaptive_model(unet_output, pseudo_maps)

                loss = criterion(adaptive_output, density_maps)
                running_loss_val += loss.item() * images.size(0)

        epoch_loss_val = running_loss_val / len(val_loader.dataset)
        val_loss_history.append(epoch_loss_val)

        print(f'Epoch {epoch}/{num_epochs - 1}, Train Loss: {epoch_loss_train:.4f}, Validation Loss: {epoch_loss_val:.4f}')
        scheduler.step(epoch_loss_val)
    return train_loss_history, val_loss_history

In [ ]:
train_loss_history = train_jointly(model_unet, model_adaptive, train_loader, val_loader, num_epochs=50)

In [ ]:
import os
from torchvision.utils import save_image

def evaluate_and_save_predictions(unet, adaptive_model, test_loader, save_dir):
    unet.eval()
    adaptive_model.eval()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        # Ajusta para ignorar el nombre del archivo
        for batch_idx, (inputs, true_density_maps, pseudo_maps, _) in enumerate(test_loader):

            inputs = inputs.to(device)
            true_density_maps = true_density_maps.to(device)
            pseudo_maps = pseudo_maps.to(device)

            predicted_density_maps_unet = unet(inputs)
            predicted_density_maps_adaptive = adaptive_model(predicted_density_maps_unet, pseudo_maps)

            for i in range(len(predicted_density_maps_adaptive)):
                pred_map = predicted_density_maps_adaptive[i].cpu().squeeze(0)
                # Usa el nombre de archivo original en lugar de 'predicted_map' y un número
                original_filename = os.path.splitext(_[i])[0] + "_predicted.png"
                save_image(pred_map, os.path.join(save_dir, original_filename))

    print(f'Predicciones guardadas en {save_dir}')

save_directory = '/content/drive/MyDrive/1 Tesis/Fish/UnetAdapt/FishPred50'

evaluate_and_save_predictions(model_unet, model_adaptive, test_loader, save_directory)

Predicciones guardadas en /content/drive/MyDrive/1 Tesis/Fish/UnetAdapt/FishPred50
